In [1]:
import numpy as np
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import json

c:\Users\61098577\Downloads\ML\label\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# The require model used 
load_model = "microsoft/DialoGPT-medium"

# download and cache tokenizer
tokenizer = AutoTokenizer.from_pretrained(load_model)

# download and cache pre-trained model
model = AutoModelForCausalLM.from_pretrained(load_model)

In [3]:
# Load the JSON file containing question-answer pairs with intents
def load_json_data(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

# Build a ChatBot class with all necessary modules to make a complete conversation
class ChatBot():

    def __init__(self, json_filepath=None):

        # Initialize chatbot history
        self.chat_history_ids = None
        self.end_chat = False
        
        # Load the JSON file for Q&A if provided
        self.qa_data = None
        if json_filepath:
            self.qa_data = load_json_data(json_filepath)
        
        # Greet when starting
        self.welcome()
        
    def welcome(self):

        print(" Welcome to Inquiry Area.......")
        time.sleep(2)
        print('Type "Namaskar" or "quit" or "exit" or "Adios" or "Auf Wiedersehen" to end chat \n')
        time.sleep(3)
        greeting = np.random.choice([
            "Welcome! I'm your chatbot companion. How can I assist you?",
            "Good to see you! I'm your virtual helper, always ready to serve.",
            "Hello! It's great to connect with you. What can I do for you?",
            "Hi! I'm here to provide you with the best assistance possible.",
            "Hi, I am a ChatBot. Let's chat!",
            "Hey! I'm your virtual assistant. Let's get started!"
        ])
        print("ChatBot >> " + greeting)
        
    def user_input(self):

        # Receive input from user
        text = input("User >> ")
        if text.lower().strip() in ['bye', 'quit', 'exit','Adios','Auf Wiedersehen','Namaskar']:
            self.end_chat = True
            print('ChatBot ----->> See you soon! Bye!')
            time.sleep(1)
            print('\n Quitting ChatBot ...')
            return None  # End the loop if chat ends
        else:
            # Preprocess the input text
            self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors='pt')
            return text

    def fetch_answer_from_intent(self, query):
        if self.qa_data:
            for intent_data in self.qa_data:
                intent = intent_data.get('intent', '')
                examples = intent_data.get('examples', [])
                response = intent_data.get('response', "Sorry, I don't have an answer to that.")
                
                # Checking the query if it matches any of the json intent using regex for case-insensitive matching
                for example in examples:
                    if re.search(re.escape(example.lower()), query.lower()):
                        return response
        return None

    def bot_response(self, user_query):
        # Try to find a response based on the user's query from JSON intent data
        response = self.fetch_answer_from_intent(user_query)
        
        if response:
            print("ChatBot >> " + response)
        else:
            # No match found in JSON, generate a response from the model
            if self.chat_history_ids is not None:
                self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
            else:
                self.bot_input_ids = self.new_user_input_ids

            # Generating a response from the model as loaded from huggingface
            self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, no_repeat_ngram_size=2)
            bot_response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

            if bot_response == "":
                bot_response = self.random_response()

            print('ChatBot >> ' + bot_response)

    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], skip_special_tokens=True)
        
        while response == '':
            i = i - 1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], skip_special_tokens=True)
        
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", "I am not sure"])
        else:
            reply = np.random.choice(["Great", "Fine. What's up?", "Okay"])
        
        return reply



In [ ]:
# Build a ChatBot object with the path to your JSON file
json_filepath = './chatbotquery.json'  # Replace with your actual file path
bot = ChatBot(json_filepath=json_filepath)

# Start chatting
while True:
    user_query = bot.user_input()
    if bot.end_chat:
        break
    bot.bot_response(user_query)

 Welcome to Inquiry Area.......
Type "Namaskar" or "quit" or "exit" or "Adios" or "Auf Wiedersehen" to end chat 

ChatBot >> Good to see you! I'm your virtual helper, always ready to serve.
ChatBot >> Hello! How can I assist you today?


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ChatBot >> I'm not sure what you're asking.
ChatBot >> We offer consulting, software development, and data analysis.
